######

### Database

- [Earth2014](http://ddfe.curtin.edu.au/models/Earth2014/) (Arc‐min shape, topography, bedrock and ice‐sheet models)




Here downloads [Earth2014.TBI2014.1min.geod.bin]

In [3]:
import numpy as np
import os
import pyshtools as pysh
#import shapefile
from osgeo import gdal
from osgeo import osr


import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
from matplotlib.colors import LightSource
from mpl_toolkits.axes_grid1 import make_axes_locatable

from qixiang import functions as fnqx

In [7]:
dir_db   = '../Data/earth2014/data_1min/topo_grids'
fname_db = 'Earth2014.TBI2014.1min.geod.bin' 
fname_save  = 'Earth2014.TBI2014.1min' 
order_db = 10800
res_deg = 1/60

# dir_db   = '../Data/earth2014/data_5min/topo_grids'
# fname_db = 'Earth2014.TBI2014.5min.geod.bin' 
# fname_save  = 'Earth2014.TBI2014.5min' 
# order_db = 2160
# res_deg = 5/60

fname_topo = os.path.join(dir_db,fname_db)

In [8]:
# This scirpt shows how to access the data grids of earth2014 model
# Source code: access_Earth2014_grids5min.m (Christian Hirt, Moritz Rexer)

# grid definitions
ylimvec = [-90,90]
xlimvec = [-180,180]

lats = np.arange((-90+res_deg/2),(90-res_deg/4),res_deg)
lons = np.arange((-180+res_deg/2),(180-res_deg/4),res_deg)
nlat = len(lats) 
nlon = len(lons)

minlon1,maxlon1,minlat1,maxlat1 = (lons.min(),lons.max(),lats.min(),lats.max())
global_extent = [minlon1,maxlon1,minlat1,maxlat1]

# read data
data_topo = np.fromfile(fname_topo, dtype='>i2').reshape((nlat, nlon))
data_topo = data_topo.astype(np.int16) # data = data.astype('<i2')
data_topo = np.flipud(data_topo)

# get SHCs
#topo = pysh.SHGrid.from_array(data_topo)
#coeffs = pysh.expand.SHExpandDH(topo.data, sampling=2)

In [9]:
# grid definitions of region
#name_area ='Tibet'
#minlon2,maxlon2,minlat2,maxlat2 = (65,110,15,45) # Tibet
name_area ='TRR'
minlon2,maxlon2,minlat2,maxlat2 = (88,108,20,36)

# name_area = 'TRR2'
# minlon2,maxlon2,minlat2,maxlat2 = (91, 106,21,35)

order_t = order_db
#minlon2,maxlon2,minlat2,maxlat2 = extent_area  

minX_index = np.int(nlon/(maxlon1-minlon1)*(minlon2-minlon1))
maxX_index = np.int(nlon/(maxlon1-minlon1)*(maxlon2-minlon1)) 
minY_index = np.int(nlat/(maxlat1-minlat1)*(minlat2-minlat1)) 
maxY_index = np.int(nlat/(maxlat1-minlat1)*(maxlat2-minlat1))

# real coordinates 
minlon3,maxlon3,minlat3,maxlat3 = (lons[minX_index],lons[maxX_index],lats[minY_index],lats[maxY_index])

In [10]:
minlon3,maxlon3,minlat3,maxlat3

(88.0083333335466, 108.00833333356252, 20.008333333327087, 36.00833333332618)

In [24]:
topo_fs = np.flipud(data_topo.copy())
data = topo_fs[minY_index:maxY_index,minX_index:maxX_index]
data = np.flipud(data)

fname_data = '../Data/dem/' + name_area + '_'+ fname_save + '.order'+ str(order_t)+ '.tif'
latRange = [minlat3,maxlat3]
lonRange = [minlon3,maxlon3]
dtype = gdal.GDT_Float32

qxfn.array2geotiff_yx(fname_data, data, latRange, lonRange, dtype)

In [22]:
gdal.GetDriverByName('GTiff').Create(fname_data, nx, ny, 1, dtype)

In [ ]:
input_file = "../Data/dem/srtm_TRR_90m.tif"
bounds = (91, 21, 106,35)
minX, minY, maxX, maxY = bounds

In [ ]:
gtiff = gdal.Open(input_file )
# width = gtiff.RasterXSize
# height = gtiff.RasterYSize
# gt = gtiff.GetGeoTransform()
img = gtiff.GetRasterBand(1).ReadAsArray()

In [ ]:
!gdalinfo $input_file

In [ ]:
fig = plt.figure(1, figsize=(5,4))
ax = fig.add_subplot(111, xlim=(minX,maxX), ylim=(minY,maxY))
# ax.axis('off')
ls = LightSource(azdeg=315, altdeg=45)
rgb = ls.shade(img, cmap=cm.terrain, blend_mode='soft', vert_exag=2., dx=50, dy=50)
im1 = ax.imshow(rgb, extent=[minX, maxX, minY, maxY], cmap=cm.terrain, origin='upper')
plt.show()

In [ ]:
# project the SRTM data for Topotoolbox input
input_file = "../Data/dem/srtm_TRR_90m.tif"
output_file = "../Data/dem/srtm_TRR_90m_lower.tif"

#!gdalinfo --version
#!gdalinfo $input_file

proj_input = 'EPSG:4326'
# lat_0 = (Ymin+Ymax)/2 lon_0 = (Xmin+Xmax)/2 
#proj_output = "'+proj=tmerc +lat_0=29 +lon_0=100 +k=1 x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'" 
proj_output = 'EPSG:4326'


!gdalwarp -s_srs $proj_input -t_srs $proj_output -ts 1400 0 -r bilinear $input_file $output_file

In [ ]:
# project the SRTM data for Topotoolbox input
input_file = "../data/dem/srtm_TRR_90m.tif"
output_file = "../data/dem/srtm_TRR_90m_lower.tif"

#!gdalinfo --version
#!gdalinfo $input_file

proj_input = 'EPSG:4326'
# lat_0 = (Ymin+Ymax)/2 lon_0 = (Xmin+Xmax)/2 
#proj_output = "'+proj=tmerc +lat_0=29 +lon_0=100 +k=1 x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'" 
proj_output = "'+proj=tmerc +lat_0=28 +lon_0=98.5 +k=1 x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'" 


!gdalwarp -s_srs $proj_input -t_srs $proj_output -ts 1400 0 -r bilinear $input_file $output_file